In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm
import numpy as np
# plt.rcParams['font.sans-serif'] = ['AR PL UKai CN']
# 读取CSV文件
# file_path = './statistics/73lasso/73lasso_stack_predictions.xlsx'  # 请替换为实际的文件路径
file_path = './final_stats/stacking_test_predictions.xlsx'
# file_path = './final_stats/voting_test_predictions_tpsa_gt_10.xlsx'
df = pd.read_excel(file_path)

# 按 'nom_score' 从大到小排序
df_sorted = df.sort_values(by='probabilities', ascending=False)


# 设置颜色，根据风险级别设置颜色
gs_colors = df_sorted['Gleason Score'].map({
    0: 'lightgreen',        # 绿色（健康）
    6: 'skyblue',        # 绿色（健康）
    7: '#eda600',        # 橙色（中风险）
    8: '#ec4241',        # 红色（高风险）
    9: '#ec4241',        # 红色（高风险）
    10: '#ec4241'        # 红色（高风险）
}).fillna('#808080')  # 用灰色代替缺失值

# 设置PSA色图和标准化（使用 PSA 值的范围 4-30）
psa_norm = Normalize(vmin=df_sorted['PSA'].quantile(0.03), vmax=df_sorted['PSA'].quantile(0.97))  # PSA值范围是 4-30
psa_colors = cm.viridis(psa_norm(df_sorted['PSA']))  # 使用 'viridis' 色图
# psa_colors = np.where(df_sorted['PSA'] < 10, '#440154', '#fee725')  # PSA小于10为蓝色，大于等于10为橙色

# PHID色图
phid_norm = Normalize(vmin=df_sorted['PHID'].min(), vmax=df_sorted['PHID'].max())  # 标准化 PHID 范围
phid_colors = cm.viridis(phid_norm(df_sorted['PHID']))  # 使用 viridis 色图

# PI-RADS色图
pirads_norm = Normalize(vmin=df_sorted['PI-RADS'].min(), vmax=df_sorted['PI-RADS'].max())  # 标准化 PI-RADS 范围
pirads_colors = cm.viridis(pirads_norm(df_sorted['PI-RADS']))  # 使用 viridis 色图

# Volume色图
volume_norm = Normalize(vmin=df_sorted['Volume'].min(), vmax=df_sorted['Volume'].max())  # 标准化 Volume 范围
volume_colors = cm.viridis(volume_norm(df_sorted['Volume']))  # 使用 viridis 色图

# 绘制瀑布图
fig, ax = plt.subplots(figsize=(12, 6), dpi=600)
bar_width = 1 
x_offset = 1  # 设置右移的偏移量
bars = ax.bar(np.arange(len(df_sorted)) + bar_width , df_sorted['probabilities'], color=gs_colors, width=bar_width)
# 去掉所有外框（spines）
for spine in ax.spines.values():
    # if spine != ax.spines['left']:  # 保留 y 轴（左侧）
    spine.set_visible(False)


# Y轴线距离
y_bias = -5
# 手动绘制 y 轴线
ax.plot([y_bias, y_bias], [0, 1], color='black', lw=1)  # 绘制从 (0, 0) 到 (0, 1) 的黑色线 
# 设置 y 轴的刻度（从 0 到 1，间隔为 0.2）
ticks = np.arange(0, 1.1, 0.2)
# 在手动绘制的 y 轴线上添加刻度及小横线
for tick in ticks:
    # 添加刻度线
    ax.plot([y_bias-0.4, y_bias], [tick, tick], color='black', lw=1)  # 小横线从 -10.1 到 -10 绘制
    # 添加刻度标签
    ax.text(y_bias-0.8, tick, f'{tick:.1f}', fontsize=8, color='black', ha='right', va='center')
 
# 在瀑布图下方添加 PSA 水平特征条
feature_bar_height = -0.05  # 控制特征条的位置（与主图的距离）
feature_bar_height_offset = -0.15  # 稍微调节特征条的高度，避免被遮挡

# 在图像下方添加 PSA, PHID, PI-RADS 颜色条
# 在瀑布图下方添加 PSA 水平特征条
feature_bar_height = -0.05  # 控制特征条的位置（与主图的距离）
feature_bar_height_offset = -0.2  # 稍微调节特征条的高度，避免被遮挡
# 添加 'PSA' 文字说明
ax.text(-1.5, feature_bar_height_offset + 0.04, 'PSA', fontsize=12, color='black', ha='right', va='center')
for i in range(len(df_sorted)):
    ax.add_patch(plt.Rectangle((i - 0.5 + x_offset, feature_bar_height_offset), bar_width, 0.1, color=psa_colors[i], linewidth=0))
    
ax.text(-1.5, feature_bar_height_offset*2 + 0.04, 'PHID', fontsize=12, color='black', ha='right', va='center')
for i in range(len(df_sorted)):
    ax.add_patch(plt.Rectangle((i - 0.5 + x_offset, feature_bar_height_offset*2), bar_width, 0.1, color=phid_colors[i], linewidth=0))

ax.text(-1.5, feature_bar_height_offset*3 + 0.04, 'PI-RADS 评分', fontsize=12, color='black', ha='right', va='center')
for i in range(len(df_sorted)):
    ax.add_patch(plt.Rectangle((i - 0.5 + x_offset, feature_bar_height_offset*3), bar_width, 0.1, color=pirads_colors[i], linewidth=0))

ax.text(-1.5, feature_bar_height_offset*4 + 0.04, '前列腺体积', fontsize=12, color='black', ha='right', va='center')
for i in range(len(df_sorted)):
    ax.add_patch(plt.Rectangle((i - 0.5 + x_offset, feature_bar_height_offset*4), bar_width, 0.1, color=volume_colors[i], linewidth=0))

# 设置横轴起点
# 设置 y 轴范围，仅限于 0 到 1，确保瀑布图的条形在这个范围内
# ax.set_yticks([0,0.2,0.4,0.6,0.8, 1])  # 只显示 0 到 1 之间的刻度
# ax.set_yticklabels([0, 0.5, 1])  # 设置对应的 y 轴标签
# plt.xlim(0, len(df_sorted) - 0.5)
plt.yticks([])
plt.xticks([])  # 隐藏横轴标签

# stacking 阈值
mean_0 = 0.2002
mean_1 = 0.9010

# voting 阈值
# mean_0 = 0.9010
# mean_1 = 0.2002

# 计算高概率样本的数量（阈值mean_1=0.8799）
n_high = len(df_sorted[df_sorted['probabilities'] > mean_1])

# 确定大括号的起始和结束x坐标（考虑条形位置和宽度）
start_x = x_offset - 0.5  # 第一个条形的左边缘
end_x = x_offset - 0.5 + n_high  # 第n_high个条形的右边缘

# 大括号的y位置（位于PSA特征条上方0.05的位置）
y_bracket = feature_bar_height_offset + 0.15 + 0.05  # PSA特征条高度0.1，再上移0.05

# 绘制横向大括号
ax.annotate('',
            xy=(start_x, y_bracket),
            xytext=(end_x, y_bracket),
            arrowprops=dict(
                arrowstyle='|-|, widthA=0.5, widthB=0.5',
                lw=1.5,
                color='black'
            ))

# 添加文字标签（位于大括号上方）
ax.text(
    (start_x + end_x) / 2,  # 文字居中
    y_bracket - 0.07,       # 稍低于大括号
    '{:.2f}%'.format(len(df_sorted[(df_sorted['labels'] == 1) & (df_sorted['probabilities'] > mean_1)])/len(df_sorted)*100),            # 文本内容
    ha='center',            # 水平居中
    va='bottom',            # 垂直对齐到底部
    fontsize=8
)
# ================== 右侧低风险区域大括号 ==================
# 计算低概率样本数量
n_low = len(df_sorted[df_sorted['probabilities'] < mean_0])
start_x_right = x_offset - 0.5 + (len(df_sorted) - n_low)
end_x_right = x_offset - 0.5 + len(df_sorted)

ax.annotate('',
            xy=(start_x_right, y_bracket),
            xytext=(end_x_right, y_bracket),
            arrowprops=dict(
                arrowstyle='|-|, widthA=0.5, widthB=0.5',
                lw=1.5,
                color='black'
            ))

ax.text(
    (start_x_right + end_x_right) / 2,
    y_bracket - 0.07,
    '{:.2f}%'.format(len(df_sorted[ (df_sorted['probabilities'] < mean_0)])/len(df_sorted)*100),
    ha='center',
    va='bottom',
    fontsize=8
)

# 添加水平的分界线（假设用某个阈值来划分风险）
threshold_prob_high = mean_1  # 设置高风险的概率阈值
threshold_prob_low = mean_0   # 设置低风险的概率阈值
# 水平分界线
plt.axhline(y=threshold_prob_high, color='black', linestyle='--', linewidth=1)
plt.axhline(y=threshold_prob_low, color='black', linestyle='--', linewidth=1)
# 在水平分界线旁边标注阈值
plt.text(len(df_sorted) * 0.2, threshold_prob_high + 0.02, f'{threshold_prob_high:.4f}', fontsize=8, color='black')
plt.text(len(df_sorted) * 0.7, threshold_prob_low + 0.02, f'{threshold_prob_low:.4f}', fontsize=8, color='black')

# 添加区域标签
# plt.text(threshold_index_high - 70, 0, 'High Risk', fontsize=12) #, va='center', ha='center')
# plt.text(threshold_index_low + 70 ,0,'Low Risk', fontsize=12) #, va='center', ha='center')
# plt.text((threshold_index_high + threshold_index_low) / 2 - 30, 0, 'Intermediate Risk', fontsize=12,) # va='center', ha='center')

# 添加图例
# plt.legend(handles=[plt.Line2D([0], [0], color='#9bef9a', lw=6, label='Benign'),
#                     plt.Line2D([0], [0], color='#9bbbe1', lw=6, label='GS=6'),
#                     plt.Line2D([0], [0], color='#eda600', lw=6, label='GS=7'),
#                     plt.Line2D([0], [0], color='#f84241', lw=6, label='GS>7')],fontsize=12)
plt.legend(handles=[plt.Line2D([0], [0], color='lightgreen', lw=6, label='良性'),
                    plt.Line2D([0], [0], color='skyblue', lw=6, label='Gleason 评分 =6'),
                    plt.Line2D([0], [0], color='#eda600', lw=6, label='Gleason 评分 =7'),
                    plt.Line2D([0], [0], color='#ec4241', lw=6, label='Gleason 评分 >7')],fontsize=12)

# 添加标题和y轴标签
# plt.title('Risk Levels')
ax.set_ylabel('Stacking 模型分数', fontsize=12)
ax.yaxis.set_label_coords(0, 0.65)  # 修改 y 坐标标签的位置
# 显示图形
plt.show()

# 统计 labels=0 且预测概率 > mean_1 的数量
count_label_0_high_prob = len(df_sorted[(df_sorted['labels'] == 0) & (df_sorted['probabilities'] > mean_1)])

# 统计 labels=1 且预测概率 < mean_0 的数量
count_label_1_low_prob = len(df_sorted[(df_sorted['labels'] == 1) & (df_sorted['probabilities'] < mean_0)])

# 输出统计结果
print(f"labels=0 且预测概率 > {mean_1} 的数量: {count_label_0_high_prob}")
print(f"labels=1 且预测概率 < {mean_0} 的数量: {count_label_1_low_prob}")

                             
print(f"labels=0 且预测概率 < {mean_0} 的数量: {len(df_sorted[(df_sorted['labels'] == 0) & (df_sorted['probabilities'] < mean_0)])}")
print(f"labels=1 且预测概率 > {mean_1} 的数量: {len(df_sorted[(df_sorted['labels'] == 1) & (df_sorted['probabilities'] > mean_1)])}")
print(f"总数：{len(df_sorted)}")
# print(len(df_sorted[df_sorted['probabilities'] > mean_1]), len(df_sorted[df_sorted['probabilities'] > mean_0])) 

### 图6 免穿刺率图

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 分组名称
groups = ['Strategy Ⅰ\n(NPV=1, PPV=1)', 
          'Strategy Ⅱ\n(NPV≥0.95, PPV=1)', 
          'Strategy Ⅲ\n(NPV≥0.95, PPV≥0.95)',
        #   'PSA 4-10\n(Low Risk)', 
        #   'PSA 10-30\n(High Risk)']
]

# 免穿刺比例（总）
avoid_biopsy_rates = [27.78+18.52, 
                      66.67, 
                      69.44]
# , 36.51+4.76, 37.78+15.56]  # 总高度

# 各分层的免穿刺构成（低风险正确分类 + 中风险误分类 + 高风险正确 + 高风险误分类）
# 格式: [正确低风险%, 中风险误分类%, 高风险误分类%]
stratified_rates = np.array([
    [27.78+18.52, 0,    0],       # Cohort 1：无误差
    [66.67-4.17,   4.17, 0],     # Cohort 2：低风险70%正确 + 中风险3.1%误分类
    [69.44-5.33,       5.33,  0],
    # [36.51+4.76,  0,    0],       # PSA 4-10
    # [37.78+15.56, 0, 0]      # PSA 10-30
])

# 误差校验（确保总和等于总免穿刺率）
# assert np.allclose(stratified_rates.sum(axis=1), avoid_biopsy_rates), "数据不匹配！"

# 颜色定义
colors = {
    'correct': 'lightgreen',    # 正确低风险（绿色）
    'misclassified': 'salmon'     # 高风险误分类（红色）
}

# 创建图表
fig, ax = plt.subplots(figsize=(8, 6), dpi=400)

# 绘制堆叠柱状图
bottom = np.zeros(len(groups))
for i, category in enumerate(['correct', 'misclassified']):
    ax.bar(groups, stratified_rates[:, i], 
           bottom=bottom, color=colors[category],
           edgecolor='black', label=category.replace('_', ' ').title(),
           width=0.5)
    bottom += stratified_rates[:, i]
    
# 添加误分类部分的数值标签（仅策略2和3）
for idx in [1, 2]:  # 仅策略Ⅱ和Ⅲ
    misclass_rate = stratified_rates[idx, 1]
    if misclass_rate > 0:
        x_pos = idx
        y_pos = stratified_rates[idx, 0] + misclass_rate / 2
        ax.text(x_pos - 0.45, y_pos,  # 向右偏移一点
                f'{misclass_rate:.2f}%', 
                ha='left', va='center',
                color='black', fontsize=10)
        
# ================= 新增部分：误分类标注 =================
# for idx in range(len(groups)):
#     # 计算误分类比例和位置
#     misclass_rate = stratified_rates[idx, 1]
#     if misclass_rate > 0:
#         # 确定标签位置（x居中，y在误分类区域中心）
#         x_pos = idx
#         y_pos = stratified_rates[idx, 0] + misclass_rate/2
        
#         # 添加数值标签
#         ax.text(x_pos, y_pos, 
#                 f'{misclass_rate:.1f}%', 
#                 ha='center', va='center',
#                 color='black', fontsize=10,
#                 # path_effects=[patheffects.withStroke(linewidth=2, foreground="black")])
#         )
        
# 添加总免穿刺率标签
for i, rate in enumerate(avoid_biopsy_rates):
    ax.text(i, rate + 1, f'Total: {rate:.1f}%', ha='center', va='bottom', fontsize=10)

# 添加误分类箭头标注（以Cohort 2为例）
# ax.annotate('Misclassified\nMedium Risk', 
#             xy=(1, 70), xytext=(1.5, 70), 
#             arrowprops=dict(arrowstyle='->', color='black', lw=1),
#             ha='left', va='center', fontsize=9)

# 图像美化
ax.set_ylabel("Avoided Biopsy (%)", fontsize=12)
ax.set_ylim(0, 100)
# ax.set_title("Avoided Biopsy Rate", fontsize=14, pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1, 1))
ax.grid(axis='y', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()